# Recap de Funciones

Los decoradores en Python son objetos que se usan para modificar funciones o clases. Para entender como funcionan los decoradores tenemos que repasar algunos aspectos importantes sobre funciones:

* El nombre que se le asigna a una funcion es una referencia a el codigo que está en esa funcion

In [1]:
def suc(x):
    return x + 1

sucesor = suc
print sucesor(10)
print suc(10)

11
11


* Podemos eliminar las referencias a una funcion sin tener que eliminar el codigo que está en esa funcion

In [2]:
del suc
print sucesor(10)

11


In [3]:
print suc(10)

NameError: name 'suc' is not defined

* Se pueden utilizar funciones dentro de funciones

In [4]:
def f():
    def g():
        print('esta es la funcion \'g\'')
        
    print('esta es la funcion \'f\'')
    print('voy a llamar a la funcion \'g\'')
    g()

f()

esta es la funcion 'f'
voy a llamar a la funcion 'g'
esta es la funcion 'g'


In [6]:
def temperatura(c):
    def celsius2fahrenheit(f):
        return 9 * f / 5 + 32

    result = str(c) + " grados Celcius es igual a " + str(celsius2fahrenheit(c)) + " grados Fahrenheit" 
    return result

print(temperatura(20))

20 grados Celcius es igual a 68 grados Fahrenheit


* Las funciones se pueden utilizar como parametros dentro de funciones

In [7]:
def g():
    print('esta es la funcion \'g\'')
    
def f(func):
    print('esta es la funcion \'f\'')
    print('voy a llamar a la funcion \'g\'')
    func()
          
f(g)

esta es la funcion 'f'
voy a llamar a la funcion 'g'
esta es la funcion 'g'


* Las funciones también pueden `return` una funcion

In [8]:
def f(x):
    def g(y):
        print 'y = ' + str(y)
        return y + x + 3
    print 'x = ' + str(x)
    return g

nf1 = f(1)
nf2 = f(3)

print(nf1(1))
print(nf2(1))

x = 1
x = 3
y = 1
5
y = 1
7


# Un simple decorador

In [9]:
def decorador(func):
    def function_wrapper(x):
        print("antes de llamar a " + func.__name__)
        func(x)
        print("despues de llamar a " + func.__name__)
    return function_wrapper

def foo(x):
    print('esta es la funcion\'foo\': ' + str(x))

print('llamamos primero a la funcion \'foo\'')
foo('hola')
    
print('ahora decoramos a \'foo\' con \'f\'')
foo = decorador(foo)

print('ahora llamamos a \'foo\' despues de decorar')
foo(42)

llamamos primero a la funcion 'foo'
esta es la funcion'foo': hola
ahora decoramos a 'foo' con 'f'
ahora llamamos a 'foo' despues de decorar
antes de llamar a foo
esta es la funcion'foo': 42
despues de llamar a foo


la declaración `foo = decorador(foo)` hace que `foo` exista dos veces en el programa: antes de la decoración y después de la decoración.

In [10]:
def decorador(func):
    def function_wrapper(x):
        print("antes de llamar a " + func.__name__)
        func(x)
        print("despues de llamar a " + func.__name__)
    return function_wrapper

@decorador
def foo(x):
    print('esta es la funcion\'foo\': ' + str(x))

foo("Hi")

antes de llamar a foo
esta es la funcion'foo': Hi
despues de llamar a foo


y podemos utilizar la declaración `@decorador` en las funciones que queramos

In [11]:
def decorador(func):
    def function_wrapper(x):
        print("antes de llamar a " + func.__name__)
        func(x) ###
        print("despues de llamar a " + func.__name__)
    return function_wrapper

@decorador
def succ(n):
    return n + 1 ###

succ(10)

antes de llamar a succ
despues de llamar a succ


también es posible decorar funciones de modulos importados

In [12]:
from math import sin, cos

def decorador(func):
    def function_wrapper(x):
        print("antes de llamar a " + func.__name__)
        res = func(x)
        print(res)
        print("despues de llamar a " + func.__name__)
    return function_wrapper

# for f in [sin, cos]:
#     print f(3.1415)

sin = decorador(sin)
cos = decorador(cos)

for f in [sin, cos]:
    f(3.1415)

antes de llamar a sin
9.26535896605e-05
despues de llamar a sin
antes de llamar a cos
-0.999999995708
despues de llamar a cos


En resumen, los decoradores en Python son objetos que se utilizan para modificar funciones, metodos o clases. 

El objeto original, el que va a ser modificado, se le pasa al decorador como un parametro. 

El decorador devuelve un objeto modificado, que esta atado al nombre que se uso en la definición.

por ahora hemos visto solamente como funcionan los decoradores utilizando funciones que solamente usan un parametro. Que pasa si queremos modificar una función que utiliza mas de un paramtero?

In [13]:
def decorador(func):
    def function_wrapper(x):
        print("antes de llamar a " + func.__name__)
        print func(x) 
        print("despues de llamar a " + func.__name__)
    return function_wrapper

@decorador
def succ(n,i):
    return n + i 

succ(10,1)

TypeError: function_wrapper() takes exactly 1 argument (2 given)

In [20]:
def decorador(func):
    def function_wrapper(x,*args):
        print("antes de llamar a " + func.__name__)
        print func(x,*args) 
        print("despues de llamar a " + func.__name__)
    return function_wrapper

@decorador
def succ(n,*args):
    if args:
        return n + sum(args)
    else:
        return n + 1 

succ(10,1,2,3,4)

antes de llamar a succ
20
despues de llamar a succ


En que casos se utilizan decoradores?

* para revisar argumentos en funciones

In [24]:
def revisar_argumentos(func):
    def wrapper(x,*args):
        assert x >= 0, 'no se pueden utilizar numeros negativos'
        print func(x,*args) 
    return wrapper

@revisar_argumentos
def succ(n,*args):
    if args:
        return n + sum(args)
    else:
        return n + 1 

succ(1)

2


* para contar cuantas veces se llama una funcion

In [25]:
def call_counter(func):
    def helper(x,*args):
        helper.calls += 1
        return func(x,*args)
    helper.calls = 0

    return helper

@call_counter
def succ(n,*args):
    if args:
        return n + sum(args)
    else:
        return n + 1 

print(succ.calls)
for i in range(10):
    succ(i,1)
    
print(succ.calls)

0
10


* para enforzar protocoles de autentificación y accesos
* para medir el tiempo que corre una funcion
* para controlar rate-limiting
* para implementar input/output logging

Los decoradores también pueden contener parametros

In [27]:
def saludo_noche(func):
    def wrapper(x):
        print "buenas noches " + func(x)
    return wrapper

def saludo_dia(func):
    def wrapper(x):
        print "buenos dias " + func(x)
    return wrapper

@saludo_noche
def saludo(x):
    return x + ", pura vida!"

saludo('Camila')

buenas noches Camila, pura vida!


In [29]:
def saludo_decorado(expr):
    def decorador(func):
        def wrapper(x):
            print expr + func(x)
        return wrapper
    return decorador
        
@saludo_decorado("buenas noches ")
def saludo(x):
    return x + ", pura vida!"

saludo('Camila')

buenas noches Camila, pura vida!


Y también se pueden utilizar más de un decorador en una funcion

In [36]:
def saludar(func):
    def wrapper(x):
        return 'Hola ' + func(x) ##
    return wrapper

def despedir(func):
    def wrapper(x):
        return func(x) + ' Adios' ##
    return wrapper

        
@saludar
@despedir
def conversar(x):
    return str(x + ", pura vida!")

conversar('Camila')

'Hola Camila, pura vida! Adios'

In [3]:
class decorador(object):

    def __init__(self, f):
        print "adentro de decorador.__init__()"
        f() # Prove that function definition has completed

    def __call__(self):
        print "adentro de decorador.__call__()"

@decorador
def foo():
    print "adentro de foo"

print "fin de decorar foo"

foo()

adentro de decorador.__init__()
adentro de foo
fin de decorar foo
adentro de decorador.__call__()
